In [33]:
!python3 -m pip install kfp --upgrade --user

You should consider upgrading via the 'C:\Users\Alika\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


### Setup


In [34]:
EXPERIMENT_NAME = 'Simple notebook pipeline'        # Name of the experiment in the UI
BASE_IMAGE = 'tensorflow/tensorflow:2.0.0b0-py3'    # Base image used for components in the pipeline

In [35]:
import kfp
import kfp.dsl as dsl
from kfp import compiler
from kfp import components

### Create pipeline component

#### Create a python function

In [36]:
@dsl.component
def add_func(a: float, b: float) -> float:
    '''Calculates sum of two arguments'''
    print(a, '+', b, '=', a + b)
    return a + b

#### Build a pipeline component from the function

### Build a pipeline using the component

In [37]:
@dsl.pipeline(
   name='Calculation pipeline',
   description='A toy pipeline that performs arithmetic calculations.'
)
def calc_pipeline(
   a: float = 0.0,
   b: float = 7.0
):
    # Passing pipeline parameter and a constant value as operation arguments
    add_task = add_func(a=a, b=4.0)  # Fix: Pass arguments as keyword arguments
    
    # You can create explicit dependency between the tasks using xyz_task.after(abc_task)
    add_2_task = add_func(a=a, b=b)  # Fix: Pass arguments as keyword arguments
    
    add_3_task = add_func(a=add_task.output, b=add_2_task.output)  # Fix: Pass arguments as keyword arguments

In [38]:
compiler.Compiler().compile(calc_pipeline, package_path='calc_pipeline.yaml')

In [41]:
# Specify pipeline argument values
arguments = {'a': 7.0, 'b': 8.0}
# Launch a pipeline run given the pipeline function definition
kfp.Client("http://127.0.0.1:8080/").create_run_from_pipeline_func(calc_pipeline, arguments=arguments, 
                                           experiment_name=EXPERIMENT_NAME)
# The generated links below lead to the Experiment page and the pipeline run details page, respectively

RunPipelineResult(run_id=9ac92f5e-84b1-4aae-a39a-32237c4bb603)